In [1]:
# Installing dependencies
import numpy as np
import pandas as pd
from yfQuery import datareader

# Custom Class

In [2]:
class DataLoader:
    '''
        Class for loading past stocks prices
        symbol: symbol can be either a single symbol or 
                a list of symbol
        start:  start date
        end:    end date
    '''
    def __init__(self, symbol, start, end):
        # Preload
        self.data = datareader(symbol, start, end)
    
    def get(self, start, end):
        # Return a period of the data
        return self.data.iloc[start: end]

In [3]:
class BuySell:
    def __init__(self, capital=None, max_share=None):
        self.original_capital = capital
        self.capital = capital
        self.max_share = max_share
        self.share = 0
        self.is_holding = False
        self.buy_at = 0
        # Array for dollar gain per trade 
        self.dollar_gain = []
        # Array for percentage gain per trade
        self.pct_gain = []
    
    def buy(self, price):
        # Buy Action
        self.buy_at = price
        self.is_holding = True
        # Buy Power with capital
        if self.capital is not None:
            self.share = self.capital // self.buy_at
            if self.max_share is not None and self.share > self.max_share:
                self.share = self.max_share
            self.capital -= np.round(self.buy_at * self.share, 2)
    
    def sell(self, price):
        # Sell Action
        self.calculate_gain(price)
        if self.capital is not None:
            self.capital += np.round(price * self.share)
            self.share = 0
        self.buy_at = 0
        self.is_holding = False
    
    def calculate_gain(self, price):
        dollar_gain = price - self.buy_at
        pct_gain = price / self.buy_at - 1
        self.dollar_gain.append(dollar_gain)
        self.pct_gain.append(pct_gain)
    
    def show_results(self):
        self.results = self.result_Series()
        print('Average gain of ${:.2f} per share after {} trades'.format(self.results['Total-Dollar-Gain'], 
                                                                self.results['Num-Trades']))
        print('and have average gain percentage of {:.2f}%'.format(self.results['Total-Percentage-Gain'] * 100))
        print('Has a {:.2f}% of good trades'.format(self.results['Percentage-of-Good-Trades'] * 100))
        if self.capital is not None:
            print('Test Ending Capital: ${:.2f} base on original capital of ${:.2f}'.format(self.capital, self.original_capital))
            print('With {:.2f}% Capital Gain'.format(self.results['Percentage-Capital-Gain'] * 100))

    def result_Series(self):
        index = ['Num-Trades', 'Total-Dollar-Gain', 'Total-Percentage-Gain', 
                 'Percentage-of-Good-Trades','Percentage-Capital-Gain']
        num_trades = len(self.dollar_gain)
        total_dollar_gain = np.mean(self.dollar_gain)
        total_pct_gain = np.mean(self.pct_gain)
        pct_good_trades = (np.array(self.dollar_gain) > 0).sum() / num_trades
        array = [num_trades, total_dollar_gain, total_pct_gain, pct_good_trades]
        if self.capital is not None:
            capital_gain = self.capital / self.original_capital - 1
            array.append(capital_gain)
            return pd.Series(array, index=index)
        return pd.Series(array, index=index[:-1])
        

# Back Testing

In [4]:
d = DataLoader('AAPL', '2018-01-01', '2019-12-31')
prices = d.data[['Close', 'Low', 'High']]

In [5]:
# MA Crossing or Conditional for possible buy
a, b = 3, 6
fast_ma = prices['Close'].rolling(a).mean()
slow_ma = prices['Low'].rolling(b).mean()
above = fast_ma > slow_ma
data = pd.concat([fast_ma, slow_ma, above], axis=1)
data.columns = ['fast_ma', 'slow_ma', 'above']
data = pd.concat([prices, data], axis=1)
data = data.dropna()

In [6]:
# Price Prediction
# Using the simple rolling return of buy low sell high to predict a good sell percentage
rolled_low = data['Low'].shift(a)
pct = data['High'] / rolled_low - 1
avg_pct = pct.mean()
avg_pct

0.022714116376167097

In [7]:
rolled_high = data['High'].shift(a)
# Adjusted High is like a padding for first requirement of a good buy price
adjusted_high = rolled_high * (1. - avg_pct) 
# Current Low Price against Adjusted High Price to get a percent different
pricing = data['Low'] / adjusted_high - 1
# Check if the percent different is lesser than the avg_pct
# which would result in a good buy of the Low price
data['Good-Buy'] = pricing <= avg_pct
data = data.dropna()

In [8]:
length = len(data)
bs = BuySell(capital=1000, max_share=1000)

for i in range(length):
    if i == 0:
        continue
    is_above = data['above'].iloc[i - 1]
    is_good_buy = data['Good-Buy'].iloc[i]
    buying_price = data['Low'].iloc[i]
    selling_pirce = data['Close'].iloc[i]
    high = data['High'].iloc[i]
    if i + 1 == length:
        if bs.is_holding:
            bs.sell(selling_pirce)
    else:
        if not bs.is_holding:
            if is_above and is_good_buy:
                bs.buy(buying_price)
        else:
            pct = high / bs.buy_at - 1
            good_sell = pct >= avg_pct
            if not is_above or good_sell:
                if good_sell:
                    bs.sell(high)
                else:
                    bs.sell(selling_pirce)


In [9]:
bs.show_results()

Average gain of $1.00 per share after 79.0 trades
and have average gain percentage of 1.99%
Has a 79.75% of good trades
Test Ending Capital: $4483.29 base on original capital of $1000.00
With 348.33% Capital Gain


In [10]:
avg_days_per_trade = length / bs.results['Num-Trades']
stock_momentum = data['Close'].iloc[-1] / data['Close'].iloc[0] - 1
print('Average days per trade: {:.2f}'.format(avg_days_per_trade))
print("Stock's momemtum: {:.2f}".format(stock_momentum))

Average days per trade: 6.30
Stock's momemtum: 0.68


# Testing without updating the Percentage Gain Mean

In [23]:
d = DataLoader('AAPL', '2020-01-01', '2020-12-31')
prices = d.data[['Close', 'Low', 'High']]

In [24]:
# MA Crossing or Conditional for possible buy
a, b = 3, 6
fast_ma = prices['Close'].rolling(a).mean()
slow_ma = prices['Low'].rolling(b).mean()
above = fast_ma > slow_ma
data = pd.concat([fast_ma, slow_ma, above], axis=1)
data.columns = ['fast_ma', 'slow_ma', 'above']
data = pd.concat([prices, data], axis=1)
data = data.dropna()

In [25]:
rolled_high = data['High'].shift(a)
# Adjusted High is like a padding for first requirement of a good buy price
adjusted_high = rolled_high * (1. - avg_pct) 
# Current Low Price against Adjusted High Price to get a percent different
pricing = data['Low'] / adjusted_high - 1
# Check if the percent different is lesser than the avg_pct
# which would result in a good buy of the Low price
data['Good-Buy'] = pricing <= avg_pct
data = data.dropna()

In [26]:
length = len(data)
bs = BuySell(capital=1000, max_share=1000)

for i in range(length):
    if i == 0:
        continue
    is_above = data['above'].iloc[i - 1]
    is_good_buy = data['Good-Buy'].iloc[i]
    buying_price = data['Low'].iloc[i]
    selling_pirce = data['Close'].iloc[i]
    high = data['High'].iloc[i]
    if i + 1 == length:
        if bs.is_holding:
            bs.sell(selling_pirce)
    else:
        if not bs.is_holding:
            if is_above and is_good_buy:
                bs.buy(buying_price)
        else:
            pct = high / bs.buy_at - 1
            good_sell = pct >= avg_pct
            if not is_above or good_sell:
                if good_sell:
                    bs.sell(high)
                else:
                    bs.sell(selling_pirce)

In [27]:
bs.show_results()

Average gain of $3.16 per share after 58.0 trades
and have average gain percentage of 3.40%
Has a 87.93% of good trades
Test Ending Capital: $6460.41 base on original capital of $1000.00
With 546.04% Capital Gain


In [28]:
avg_days_per_trade = length / bs.results['Num-Trades']
stock_momentum = data['Close'].iloc[-1] / data['Close'].iloc[0] - 1
print('Average days per trade: {:.2f}'.format(avg_days_per_trade))
print("Stock's momemtum: {:.2f}".format(stock_momentum))

Average days per trade: 4.28
Stock's momemtum: 0.71
